In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the image
img = plt.imread("galaxies.png")

# convert the image to float from 0 to 1 from 0 to 255
red, green, blue = [img[:, :, j] for j in range(3)]

# perform the SVD on each channel
Ur, Sr, Vr = np.linalg.svd(red, full_matrices=False)
Ug, Sg, Vg = np.linalg.svd(green, full_matrices=False)
Ub, Sb, Vb = np.linalg.svd(blue, full_matrices=False)

sg = np.diag(Sg)
sr = np.diag(Sr)
sb = np.diag(Sb)
j, j = sg.shape
imgx, imgy, imgc = img.shape

# 2-norm of each channel to compute relative error
norm_red, norm_green, norm_blue = np.linalg.norm(red, 2), np.linalg.norm(green, 2), np.linalg.norm(blue, 2)

ranks = []
frobenius_error = []
norm2_error = []
norm2_rel_error = []
rank = 10
root_singular_values_sqrd = []
sigma_v_plus_one = []
max_rank = 10
while rank <= max_rank:
    # Image reconstruction
    red_re = np.matmul(Ur[:, :rank], np.matmul(sr[:rank, :rank], Vr[:rank, :]))
    green_re = np.matmul(Ug[:, :rank], np.matmul(sg[:rank, :rank], Vg[:rank, :]))
    blue_re = np.matmul(Ub[:, :rank], np.matmul(sb[:rank, :rank], Vb[:rank, :]))
    img_re = np.stack([red_re, green_re, blue_re], axis=-1)

    # Saving the Image
    plt.imshow(img_re)
    plt.title("Reconstructed Image for rank {}".format(rank), fontsize=14)
    # plt.imsave("Recon-Img/galaxies-reconstructed-{}.png".format(rank), img_re)
    plt.savefig("Recon-Img/galaxies-reconstructed-{}.png".format(rank))
    # plt.show()

    # 2-norm Error
    error = [np.linalg.norm(red - red_re, 2), np.linalg.norm(green - green_re, 2), np.linalg.norm(blue - blue_re, 2)]
    norm2_error.append(error)
    norm2_rel_error.append([error[0]/norm_red, error[1]/norm_green, error[2]/norm_blue])

    # Frobenius Error
    frob_error = [np.linalg.norm(red - red_re, "fro"), np.linalg.norm(green - green_re, "fro"), np.linalg.norm(blue - blue_re, "fro")]
    frobenius_error.append(frob_error)      

    # Square root of sum of squares of singular values for red, green, blue channels
    root_singular_values_sqrd.append([np.sqrt(np.sum(np.square(Sr[rank:]))), np.sqrt(np.sum(np.square(Sg[rank:]))), np.sqrt(np.sum(np.square(Sb[rank:])))])
    
    # Sigma v+1 for red, green, blue channels 
    sigma_v_plus_one.append([Sr[rank], Sg[rank], Sb[rank]])
    
    ranks.append(rank)
    
    if rank < 200:
        rank = rank + 10
    elif rank < 500:
        rank = rank + 50
    else:
        rank = 2 * rank 


# Plots for SVD of original matrix all three channels in semi-log scale y axis with fontsize for labels and title 14
plt.plot(np.diag(sr), label="Red Channel", color="red", linewidth=2)
plt.plot(np.diag(sg), label="Green Channel", color="green", linewidth=2)
plt.plot(np.diag(sb), label="Blue Channel", color="blue", linewidth=2)
plt.yscale("log")
plt.xlabel("Singular Value Number", fontsize=14)
plt.ylabel("Magnitude of Singular Value (log scale)", fontsize=14)
plt.title("Singular Values of Original Matrix", fontsize=14)
plt.legend(fontsize=14)
plt.savefig("Singular-Values-Original-Matrix.png")
plt.show()


# Plot relative error for each channel vs rank
# Not used as the values were tabulated in the report
plt.plot(ranks, [norm2_rel_error[i][0] * 100 for i in range(len(ranks))], label="Red Channel", color="red", linewidth=2)
plt.plot(ranks, [norm2_rel_error[i][1] * 100 for i in range(len(ranks))], label="Green Channel", color="green", linewidth=2)
plt.plot(ranks, [norm2_rel_error[i][2] * 100 for i in range(len(ranks))], label="Blue Channel", color="blue", linewidth=2)
plt.xlabel("Rank", fontsize=14)
plt.ylabel("Percentage Relative Error (2-Norm)", fontsize=14)
plt.title("Relative Error vs Rank", fontsize=14)
plt.xticks([i for i in range(0, ranks[-1], 50)])
plt.legend(fontsize=14)
plt.savefig("Relative-Error-vs-Rank.png")
plt.show()


# LaTeX tabularx format:
def latex():
    # For red channel
    print("For red channel:\n")
    print("Rank & 2-norm error &  $\sigma_{v+1}$ & Frobenius error & & $Sum Sqr sigma$\\\\")
    for i in range(len(ranks)):
        print("\\hline")
        print("{} & {:.4f} & {:.4f} & {:.4f} & {:.4f} \\\\".format(ranks[i], norm2_error[i][0], sigma_v_plus_one[i][0], frobenius_error[i][0], root_singular_values_sqrd[i][0]))
    print("\\hline")    

    # For green channel
    print("\n\nFor green channel:\n")
    print("Rank & 2-norm error &  $\sigma_{v+1}$ & Frobenius error & & $Sum Sqr sigma$\\\\")
    for i in range(len(ranks)):
        print("\\hline")
        print("{} & {:.4f} & {:.4f} & {:.4f} & {:.4f} \\\\".format(ranks[i], norm2_error[i][1], sigma_v_plus_one[i][1], frobenius_error[i][1], root_singular_values_sqrd[i][1]))
    print("\\hline")   

    # For blue channel
    print("\n\nFor blue channel:\n")
    print("Rank & 2-norm error &  $\sigma_{v+1}$ & Frobenius error & & $Sum Sqr sigma$\\\\")
    for i in range(len(ranks)):
        print("\\hline")   
        print("{} & {:.4f} & {:.4f} & {:.4f} & {:.4f} \\\\".format(ranks[i], norm2_error[i][2], sigma_v_plus_one[i][2], frobenius_error[i][2], root_singular_values_sqrd[i][2]))
    print("\\hline")   

    # rank, red rel error, green rel error, blue rel error
    print("Rank & Red Channel & Green Channel & Blue Channel\\\\")
    for i in range(len(ranks)):
        print("\\hline")   
        print("{} & {:.4f} & {:.4f} & {:.4f} \\\\".format(ranks[i], norm2_rel_error[i][0] * 100, norm2_rel_error[i][1] * 100, norm2_rel_error[i][2] * 100))
    print("\\hline")

